In [14]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
from sklearn.preprocessing import OneHotEncoder

In [4]:
path = r'Acted Emotional Speech Dynamic Database'
list_path = []
for index,name in enumerate(os.listdir(path)):
    folder = os.path.join(path,name)
    for file_class in os.listdir(folder):
        in_folder = os.path.join(folder, file_class)
        list_path.append([in_folder,index])
    print(name,index)

fear 0
sadness 1
happiness 2
anger 3
disgust 4


In [5]:
df = pd.DataFrame(list_path,columns=['Path','label'])

In [6]:
df.head()

,Path,label
0,Acted Emotional Speech Dynamic Database/fear/f...,0
1,Acted Emotional Speech Dynamic Database/fear/f...,0
2,Acted Emotional Speech Dynamic Database/fear/f...,0
3,Acted Emotional Speech Dynamic Database/fear/f...,0
4,Acted Emotional Speech Dynamic Database/fear/f...,0


In [7]:
def cal_mfcc(path):
    #print(path)
    X,sample_rate = librosa.load(path,res_type = 'kaiser_fast')
    mfcc_mean = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    return mfcc_mean

In [8]:
mfcc = df['Path'].apply(lambda x:cal_mfcc(x))

In [12]:
X = [x for x in mfcc]
X = np.array(X)
X.shape

(604, 40)

In [13]:
X = np.expand_dims(X,-1)
X.shape

(604, 40, 1)

In [17]:
one_hot_label = OneHotEncoder()
y = one_hot_label.fit_transform(df[['label']])
y = y.toarray()
y.shape

(604, 5)

In [64]:
#simple

In [55]:
X_t = np.array(mfcc.tolist())

In [100]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_t,y,test_size=0.25,random_state=23)

# LSTM

In [113]:
model = Sequential([
    LSTM(256,input_shape=(40,1)),
    Dense(128,activation='relu'),
    Dropout(0.2),
    Dense(64,activation='relu'),
    Dropout(0.2),
    Dense(5,activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 256)               264192    
_________________________________________________________________
dense_97 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_36 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_98 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_37 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_99 (Dense)             (None, 5)                 325       
Total params: 305,669
Trainable params: 305,669
Non-trainable params: 0
_______________________________________________

In [114]:
history = model.fit(X,y,validation_data=(X_test,Y_test),epochs=50,batch_size=32)

Epoch 1/50
19/19 [==============================] - 2s 68ms/step - loss: 1.5875 - accuracy: 0.2252 - val_loss: 1.5402 - val_accuracy: 0.2314
Epoch 2/50
19/19 [==============================] - 1s 52ms/step - loss: 1.5459 - accuracy: 0.2897 - val_loss: 1.4860 - val_accuracy: 0.3388
Epoch 3/50
19/19 [==============================] - 1s 52ms/step - loss: 1.4867 - accuracy: 0.3411 - val_loss: 1.3866 - val_accuracy: 0.3719
Epoch 4/50
19/19 [==============================] - 1s 53ms/step - loss: 1.4471 - accuracy: 0.3377 - val_loss: 1.4127 - val_accuracy: 0.3306
Epoch 5/50
19/19 [==============================] - 1s 54ms/step - loss: 1.4086 - accuracy: 0.3874 - val_loss: 1.3157 - val_accuracy: 0.3802
Epoch 6/50
19/19 [==============================] - 1s 54ms/step - loss: 1.3502 - accuracy: 0.4139 - val_loss: 1.2862 - val_accuracy: 0.4545
Epoch 7/50
19/19 [==============================] - 1s 55ms/step - loss: 1.3441 - accuracy: 0.3940 - val_loss: 1.2580 - val_accuracy: 0.4876
Epoch 8/50
19

In [117]:
score = model.evaluate(X_test,Y_test, verbose = 1) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])
score = model.evaluate(X_train, Y_train, verbose = 1) 

print('Train loss:', score[0]) 
print('Train accuracy:', score[1])

4/4 [==============================] - 0s 25ms/step - loss: 0.3947 - accuracy: 0.8595
Test loss: 0.39469823241233826
Test accuracy: 0.8595041036605835
16/16 [==============================] - 0s 17ms/step - loss: 0.3731 - accuracy: 0.8468
Train loss: 0.37310490012168884
Train accuracy: 0.8467909097671509


In [120]:
y_predict = model.predict(X_test)
y_predict_bool = np.argmax(y_predict,axis=1).tolist()
y_test_e = np.argmax(Y_test,axis=1).tolist()

In [122]:
from sklearn.metrics import precision_score,recall_score,confusion_matrix,classification_report
print(confusion_matrix(y_test_e,y_predict_bool))

[[15  0  3  0  4]
 [ 2 26  0  0  0]
 [ 1  0 23  2  0]
 [ 0  0  1 22  0]
 [ 1  0  3  0 18]]


In [123]:
print(classification_report(y_test_e,y_predict_bool))

              precision    recall  f1-score   support

           0       0.79      0.68      0.73        22
           1       1.00      0.93      0.96        28
           2       0.77      0.88      0.82        26
           3       0.92      0.96      0.94        23
           4       0.82      0.82      0.82        22

    accuracy                           0.86       121
   macro avg       0.86      0.85      0.85       121
weighted avg       0.86      0.86      0.86       121

